<a href="https://colab.research.google.com/github/hyngon90/StatQuestTutorial/blob/main/06_Encoder_Decoder_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

In [ ]:
'''
Translater English to Spanish

let's go
vamos

let's go <EOS> vamos <EOS>

'''

"\nTranslater English to Spanish\n\nlet's go\nvamos\n\nlet's go <EOS> vamos <EOS>\n\n"

In [ ]:
class PositionEncoding(nn.Module):
  def __init__(self, d_model, max_len):
    super().__init__()

    pe = torch.zeros(max_len, d_model)

    pos = torch.arange(start = 0, end = max_len, step=1).float().unsqueeze(1)
    idx = torch.arange(start = 0, end = d_model, step=2).float()

    div = 1/torch.tensor(10000.0)**(idx / d_model)

    pe[:, 0::2] = torch.sin(pos * div)
    pe[:, 1::2] = torch.cos(pos * div)

    self.register_buffer('pe', pe)

  def forward(self, word_embeddings):
    return word_embeddings + self.pe[:word_embeddings.size(0), :]

In [ ]:
class Attention(nn.Module):
  def __init__(self, d_model):
    super().__init__()

    self.row_dim = 0
    self.col_dim = 1

    self.W_q = nn.Linear(in_features = d_model, out_features = d_model, bias = False)
    self.W_k = nn.Linear(in_features = d_model, out_features = d_model, bias = False)
    self.W_v = nn.Linear(in_features = d_model, out_features = d_model, bias = False)

  def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask = None):
    q = self.W_q(encodings_for_q)
    k = self.W_k(encodings_for_k)
    v = self.W_v(encodings_for_v)

    sims = torch.matmul(q, k.transpose(dim0 = self.row_dim , dim1=self.col_dim))
    scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

    if mask is not None:
      scaled_sims = scaled_sims.masked_fill(mask = mask, value=-1e9)

    attention_percents = F.softmax(scaled_sims, dim = self.col_dim)
    attention_score = torch.matmul(attention_percents, v)

    return attention_score

In [ ]:
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()

    self.we = nn.Embedding(num_embedding = num_encoder_token, embedding_dim = d_model)
    self.pe = PositionEncoding(d_model = d_model, max_len = max_len)
    self.self_attention = Attention(d_model = d_model)

  def forward(self, encoder_token_ids):
    word_embeddings = self.we(encoder_token_ids)
    position_encoded = self.pe(word_embeddings)

    self_attention_values = self.self_attention(position_encoded, position_encoded, position_encoded)
    residual_connection_values = position_encoded + self_attetion_valules

    return residual_connection_values